In [ ]:
import hsfs
import pandas as pd
conn = hsfs.connection()
fs = conn.get_feature_store()

In [ ]:
inputs = {
    'cc_num': 4738877982476642,
    'amount': 123,
    'category': 'Cash Withdrawal',
    'country': 'US'
}

In [ ]:
fraud_fv = fs.get_feature_view("fraud_model", 1)
profiles_country_fv = fs.get_feature_view("profiles_country", 1)

In [ ]:
from fraud.features.transactions.transactions import transaction_abroad

In [ ]:
def transaction_abroad(issuer_country: pd.Series, transaction_country: pd.Series) -> pd.Series:
    return (issuer_country != transaction_country).replace({True: 1, False: 0})

In [ ]:
country = profiles_country_fv.get_feature_vector(entry={'cc_num': 4738877982476642})[0]

In [ ]:
country

In [ ]:
is_transaction_abroad = transaction_abroad(pd.Series(data=inputs['country']), pd.Series(data=country))

In [ ]:
is_transaction_abroad

In [ ]:
feature_vector = fraud_fv.get_feature_vector(entry={'cc_num': 4738877982476642},
                      passed_features={
                          'amount': 123,
                          'category': 'Cash Withdrawal',
                          'is_transaction_abroad': is_transaction_abroad[0],
                      },
                      return_type="pandas",
                )

feature_vector['cc_num'] = 4738877982476642
feature_vector['event_time'] = pd.Timestamp.now()
feature_vector['prediction'] = 0

In [ ]:
fg = fs.create_feature_group(
    name="monitoring_fraud_model",
    version=2,
    description="Feature group monitoring fraud model deployment",
    primary_key=['cc_num'],
    event_time="event_time",
    statistics_config=True,
)

In [ ]:
fg.save(feature_vector)